In [31]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime
import glob

In [32]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [33]:
def find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file

def clean_position(position):
    position_copy = position.copy()
    position_copy = position_copy[
        position_copy["Current Value"].notna()
    ]  # remove rows without current value
    position_copy["Current Value"] = transfer_dollar_to_float(
        position_copy["Current Value"]
    )
    position_copy["Cost Basis Total"] = transfer_dollar_to_float(
        position_copy["Cost Basis Total"]
    )
    return position_copy

def transfer_dollar_to_float(dat):
    """
    Change "$123,456" to 123455
    """
    return dat.str.replace("$", "", regex=False).astype(float)


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)

    transactions = combine_transaction_files(transaction_files)
    transactions = clean_transactions(transactions)
    return transactions

def combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(13)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions


def clean_transactions(transactions):
    transactions_copy = transactions.copy()
    transactions_copy = transactions_copy[
        transactions_copy["Amount ($)"].notna()
    ]  # remove rows without  value
    transactions_copy["Run Date"] = pd.to_datetime(
        transactions_copy["Run Date"], format=" %m/%d/%Y"
    )
    transactions_copy["Settlement Date"] = pd.to_datetime(
        transactions_copy["Settlement Date"], format="%m/%d/%Y"
    )
    transactions_copy.loc[transactions_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    transactions_copy["Symbol"] = transactions_copy[
        "Symbol"
    ].str.lstrip()  # remove space at the beginning of Symbol
    transactions_copy = transactions_copy.sort_values(by="Run Date").reset_index(
        drop=True
    )
    return transactions_copy

In [34]:
## load position
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'


In [35]:
position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
position_files = glob.glob(position_file_path_pattern)
position_file = find_latest_position_file(position_files)
position = pd.read_csv(position_file)
position = clean_position(position)

In [36]:
transaction_file_path_pattern = os.path.join(
    data_folder_path, transaction_file_pattern
)
transaction_files = glob.glob(transaction_file_path_pattern)

transactions = combine_transaction_files(transaction_files)
transactions = clean_transactions(transactions)
print(f"The latest transaction date is {transactions['Run Date'].max()}")

The latest transaction date is 2025-03-06 00:00:00


In [78]:
class Portfolio:
    
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        
        self.today = datetime.now().date()
        self.individual_transactions = self.transactions[
            self.transactions["Account"].isin(["Individual Z23390746","Individual"]) 
        ]
        self.pension_transactions = self.transactions[
            self.transactions["Account"].isin(["ERNST & YOUNG 401(K) 86964","ERNST & YOUNG 401(K)"]) 
        ]
        self.HSA_transactions = self.transactions[
            self.transactions["Account"].isin(["Health Savings Account 241802439","Health Savings Account"]) 
        ]
        self.cash_transactions = self.transactions[
            self.transactions["Account"].isin(["Cash Management (Individual) Z06872898","Cash Management (Individual)"])
        ]
        
        self.individual_position = self.position[
            self.position["Account Number"].isin(["Z23390746"])
        ]
        self.pension_position = self.position[
            self.position["Account Number"].isin(["86964"])
        ]
        self.HSA_position = self.position[
            self.position["Account Number"].isin(["241802439"])
        ]
        self.cash_position = self.position[
            self.position["Account Number"].isin(["Z06872898"])
        ]
        
    def show_individual_investment_distribution(self):
        individual_investment_distribution = self.get_individual_investment_distribution()
        individual_investment_distribution["Percent"] = [
            f"{x * 100:.2f}%" for x in individual_investment_distribution["Percent"]
        ]
        print(pd.DataFrame(individual_investment_distribution))
        
    def get_individual_investment_distribution(self):
        total_mv = self.get_individual_mv()
        stock_mv = self.get_stock_mv()
        bill_mv = self.get_bill_mv()
        cash_mv = self.get_cash_mv()
        result = {
            "Class": ["stock", "bill", "cash", "total"],
            "Amount": [
                stock_mv,
                bill_mv,
                cash_mv,
                total_mv,
            ],
            "Percent": [
                stock_mv / total_mv,
                bill_mv / total_mv,
                cash_mv / total_mv,
                total_mv / total_mv,
            ],
        }

        return result
    
    def get_total_investment(self):
        total_investment = self.individual_transactions[
            self.individual_transactions["Symbol"] == "Transfer"
        ]["Amount ($)"].sum()
        return total_investment
    
    def get_cash_mv(self):
        cash_index = self.individual_position['Description']=='HELD IN MONEY MARKET'
        return self.individual_position[cash_index]['Current Value'].sum()
    
    def get_bill_mv(self):
        bill_index = self.individual_position['Description'].str.contains("UNITED STATES TREAS BILLS")
        return self.individual_position[bill_index]['Current Value'].sum()
    
    def get_stock_mv(self):
        cash_index = self.individual_position['Description']=='HELD IN MONEY MARKET'
        bill_index = self.individual_position['Description'].str.contains("UNITED STATES TREAS BILLS")
        return self.individual_position[(~cash_index)&(~bill_index)]['Current Value'].sum()
    
    def get_individual_mv(self):
        return self.individual_position['Current Value'].sum()

In [79]:
my_prot = Portfolio(transactions,position)

In [80]:
my_prot.show_individual_investment_distribution()

   Class      Amount  Percent
0  stock   333113.17   31.90%
1   bill   507480.30   48.60%
2   cash   203687.13   19.51%
3  total  1044280.60  100.00%


In [56]:
position["Account Number"].unique()

array(['Z23390746', '86964', 'Z06872898', '241802439'], dtype=object)

In [74]:
my_prot.individual_position

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,Z23390746,Individual,FZFXX**,HELD IN MONEY MARKET,NaN,NaN,NaN,203687.13,NaN,NaN,NaN,NaN,19.51%,NaN,NaN,Cash
1,Z23390746,Individual,AMZN,AMAZON.COM INC,10.000,$200.70,-$7.66,2007.00,-$76.60,-3.68%,+$1057.00,+111.26%,0.19%,950.00,$95.00,Cash
2,Z23390746,Individual,JPM,JPMORGAN CHASE &CO. COM,10.000,$246.54,-$4.99,2465.40,-$49.90,-1.99%,+$1265.40,+105.45%,0.24%,1200.00,$120.00,Cash
3,Z23390746,Individual,GOOGL,ALPHABET INC CAP STK CL A,20.000,$172.35,-$0.67,3447.00,-$13.40,-0.39%,+$1617.00,+88.36%,0.33%,1830.00,$91.50,Cash
4,Z23390746,Individual,AXP,AMERICAN EXPRESS CO COM USD0.20,10.000,$275.64,-$8.54,2756.40,-$85.40,-3.01%,+$1256.40,+83.76%,0.26%,1500.00,$150.00,Cash
5,Z23390746,Individual,AAPL,APPLE INC,70.000,$235.33,-$0.41,16473.10,-$28.70,-0.18%,+$5713.11,+53.09%,1.58%,10759.99,$153.71,Cash
6,Z23390746,Individual,TSLA,TESLA INC COM,20.000,$263.45,-$15.65,5269.00,-$313.00,-5.61%,+$1501.50,+39.85%,0.50%,3767.50,$188.38,Cash
7,Z23390746,Individual,SBUX,STARBUCKS CORP COM USD0.001,30.000,$105.47,-$6.22,3164.10,-$186.60,-5.57%,+$710.70,+28.96%,0.30%,2453.40,$81.78,Cash
8,Z23390746,Individual,COKE,COCA COLA CONS INC COM,2.000,$1337.55,-$18.79,2675.10,-$37.58,-1.39%,+$500.10,+22.99%,0.26%,2175.00,$1087.50,Cash
9,Z23390746,Individual,MCD,MCDONALD S CORP,20.000,$310.37,+$2.92,6207.40,+$58.40,+0.94%,+$1049.90,+20.35%,0.59%,5157.50,$257.88,Cash
